# Tutorial GeoPy

Para instalar a biblioteca, utilize o comando abaixo. (Remova o comentário)

In [38]:
#!pip install geopy
#!pip install geobr

In [1]:
import pandas as pd
import geopy

A título de exemplo, vamos utilizar os dados das sedes municipais que podem ser obtidos utilizado o código abaixo. 

*As bibliotecas "pandas" e "geopy" também devem estar instaladas para execução dos testes.

In [11]:
from geobr import read_municipal_seat

In [12]:
sedes_municipais = read_municipal_seat(year=2010)

In [13]:
sedes_municipais[:3]

,code_muni,name_muni,code_state,abbrev_state,code_region,name_region,year,geometry
0,1100015.0,Alta Floresta D'oeste,11,RO,1,Norte,2010,POINT (-61.99982 -11.93554)
1,1100023.0,Ariquemes,11,RO,1,Norte,2010,POINT (-63.03327 -9.90846)
2,1100031.0,Cabixi,11,RO,1,Norte,2010,POINT (-60.54431 -13.49976)


O código abaixo está filtrando a base de dados de modo a para obter sedes municipais que estão em Minas Gerais.

In [14]:
mg = sedes_municipais[sedes_municipais['abbrev_state'] == 'MG']

In [15]:
mg[:3]

,code_muni,name_muni,code_state,abbrev_state,code_region,name_region,year,geometry
2243,3100104.0,Abadia Dos Dourados,31,MG,3,Sudeste,2010,POINT (-47.39683 -18.48756)
2244,3100203.0,Abaeté,31,MG,3,Sudeste,2010,POINT (-45.44619 -19.15585)
2245,3100302.0,Abre Campo,31,MG,3,Sudeste,2010,POINT (-42.48099 -20.30145)


**Tarefas que podem ser feitas com o GeoPy**



1.   Calcular a distância(geodésica ou esférica/) entre dois locais
2.   Geocodificação



Obtendo as coordenadas de latitude e longitude das sedes municipais de Minas Gerais.

**Lembrando que as coordenadas colocadas na coluna “geometry” estão na ordem “Longitude, Latitude”.

In [16]:
long_lat = []
saida = mg.apply(lambda x: long_lat.append([x['geometry'].x, x['geometry'].y]), 1).dropna()
sedes_coord = {}

Antes de começar a explicar o funcionamento dos métodos, o código abaixo cria um dicionário com os nomes das sedes municipais de MG juntamente com suas coordenadas.("Longitude, Latitude", nessa ordem)

In [17]:
for i in range(0, len(long_lat)):
  sedes_coord[list(mg['name_muni'])[i]] = long_lat[i]


### 1 - Cálculo da Distância entre dois locais

In [18]:
local1 = sedes_coord['Abadia Dos Dourados']
local2 = sedes_coord['Belo Horizonte']
print(local1, local2)

[-47.396832440622156, -18.487564958938947] [-43.92645317353045, -19.93752429377505]


In [19]:
from geopy import distance
distancia = distance.distance(local1, local2)
print(distancia)

401.90576190255285 km


A unidade de saída padrão é o “km”, mas podemos mostrá-la em milhas. (Caso for necessário)

In [20]:
distancia = distance.distance(local1, local2).miles
distancia

249.73266244044333

O forma de medida padrão é a distância geodésica, mas podemos escolher a distância esférica da seguinte forma.

*A diferença entre as duas é que a distância geodésica se aplica a superfícies gerais que tenham curvatura, como: Esferas e Elipsoides. 
Enquanto que a distância esférica se aplica apenas a superfícies esféricas. Toda distância esférica é geodésica, mas nem toda geodésica é esférica.

In [21]:
from geopy import distance
distancia = distance.great_circle(local1, local2).km
distancia

401.98397357091784

No cálculo da distância geodésica, existem vários modelos que podem ser seguidos. O padrão é o WGS-84 (World Geodetic System - 1984), que é o mais preciso, mas o Geopy também suporta outros como: GRS-80, Airy (1830), Clarke (1880), GRS-67. Eles podem ser alterados pelo parâmetro “ellipsoid” do método de distância geodésica. 

In [ ]:
distancia = distance.geodesic(local1, local2, ellipsoid='Clarke (1880)')
distancia

Distance(401.9041497954684)

###  2 - Geocodificação 

In [23]:
from geopy import Nominatim
geolocator = Nominatim(user_agent="tutorial_cda") 

In [36]:
estadio = geolocator.geocode("Mineirão")
estadio

Location(Estádio Governador Magalhães Pinto, 1001, Avenida Antônio Abrahão Caram, São José, São Luiz, Pampulha, Belo Horizonte, Microrregião Belo Horizonte, Região Metropolitana de Belo Horizonte, Região Geográfica Intermediária de Belo Horizonte, Minas Gerais, Região Sudeste, 31275-000, Brasil, (-19.86588775, -43.97106904150964, 0.0))

In [37]:
reverso = geolocator.reverse("-19.86588775, -43.97106904150964, 0.0")
reverso

Location(Estádio Governador Magalhães Pinto, 1001, Avenida Antônio Abrahão Caram, São José, São Luiz, Pampulha, Belo Horizonte, Microrregião Belo Horizonte, Região Metropolitana de Belo Horizonte, Região Geográfica Intermediária de Belo Horizonte, Minas Gerais, Região Sudeste, 31275-000, Brasil, (-19.86588775, -43.97106904150964, 0.0))